 Cell 1: 專案說明與環境設定

In [ ]:
# ==============================================================================
# Cell 1: 專案說明與環境設定
# ==============================================================================
#
# 專案目標：
# 驗證 '語義相似度' (Semantic Similarity) 作為一個機器學習特徵，
# 是否比 '字元級 Jaccard 相似度' (Character-level Jaccard Similarity) 
# 更能有效地區分不同勝負標籤 (A勝/B勝/平手) 的數據。
#
# 工作流程：
# 1. (Cell 2) 加載 train.csv 數據，並預先計算 'label' 和 'resp_jaccard' 等基礎欄位。
# 2. (Cell 3) 從本地加載 GloVe 詞向量模型，並定義計算語義相似度的核心函數。
# 3. (Cell 4) 將語義相似度函數應用於數據，生成新的 'semantic_similarity' 特徵。
# 4. (Cell 5) 透過視覺化圖表與統計數據，並排對比兩種相似度特徵的有效性。
#
# --- 載入所有必要的函式庫 ---
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- Matplotlib 中文與負號顯示設定 (美化圖表) ---
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Heiti TC', 'sans-serif'] # 優先使用微軟正黑體
plt.rcParams['axes.unicode_minus'] = False

# --- tqdm 與 pandas 整合 ---
tqdm.pandas()

print("--- Cell 1: 環境設定完畢 ---")

Cell 2: 數據加載與基礎特徵準備

In [ ]:
# ==============================================================================
# Cell 2: 數據加載與基礎特徵準備
# 說明：此 Cell 負責加載您的數據，並創建後續分析所需的 'df' DataFrame
# 和 'label'、'resp_jaccard' 欄位。請務必先執行此 Cell。
# ==============================================================================
print("--- Cell 2: 開始加載和準備基礎數據 ---")

try:
    # 從 CSV 文件加載
    file_path = 'train.csv'
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"找不到數據文件: {file_path}")
        
    df = pd.read_csv(file_path)
    print(f"數據 '{file_path}' 加載成功，形狀為: {df.shape}")

    # --- 確保後續分析所需的欄位存在 ---

    # 1. 創建 'label' 欄位 (0: A勝, 1: B勝, 2: 平手)
    if 'label' not in df.columns:
        print("正在創建 'label' 欄位...")
        def get_label(row):
            if row['winner_model_a'] == 1: return 0
            if row['winner_model_b'] == 1: return 1
            return 2
        df['label'] = df.apply(get_label, axis=1)

    # 2. 創建 'resp_jaccard' 欄位以供對比
    if 'resp_jaccard' not in df.columns:
        print("正在創建 'resp_jaccard' 欄位...")
        def get_char_jaccard_similarity(text1, text2):
            set1, set2 = set(str(text1)), set(str(text2))
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return intersection / union if union != 0 else 0
        df['resp_jaccard'] = df.progress_apply(lambda row: get_char_jaccard_similarity(row['response_a'], row['response_b']), axis=1)

    print("\n'df' 已成功創建並準備就緒，包含 'label' 和 'resp_jaccard' 欄位。")
    # 顯示 DataFrame 的前幾行以作確認
    display(df.head(3))

except FileNotFoundError as e:
    print(f"\n錯誤：{e}\n請確認 'train.csv' 檔案與您的筆記本在同一個資料夾中。")
except Exception as e:
    print(f"發生未知錯誤: {e}")

Cell 3: GloVe 模型加載與核心函數定義

In [ ]:
# ==============================================================================
# Cell 3: GloVe 模型加載與核心函數定義
# 說明：此 Cell 從本地加載 GloVe 模型並定義相關計算函數。
# ==============================================================================
print("--- Cell 3: 準備 GloVe 模型與相關工具 ---")

# --- 定義本地文件路徑並進行檢查 ---
glove_file_path = 'glove.6B.100d.txt'

if not os.path.exists(glove_file_path):
    raise FileNotFoundError(
        f"錯誤：在當前目錄下找不到 '{glove_file_path}'。\n"
        "請確認您已手動下載該文件，並將其與您的 .ipynb 文件放在同一個資料夾中。"
    )
print(f"成功找到本地 GloVe 文件: '{glove_file_path}'")

# --- 準備 NLTK 工具 (停用詞、分詞器) ---
try:
    stopwords.words('english')
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("正在下載 NLTK 依賴項 (stopwords, punkt)...")
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    print("NLTK 依賴項準備完畢。")
stop_words = set(stopwords.words('english'))

# --- 定義核心計算函數 ---
def load_glove_embeddings(file_path):
    print(f"正在從 '{file_path}' 加載 GloVe 詞向量...")
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="加載詞向量"):
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                pass
    print(f"成功加載 {len(embeddings_index)} 個詞向量。")
    return embeddings_index

def sentence_to_vector(sentence, embeddings, dim=100):
    if not isinstance(sentence, str): return np.zeros(dim)
    words = word_tokenize(sentence.lower())
    valid_words = [word for word in words if word.isalpha() and word not in stop_words and word in embeddings]
    if not valid_words: return np.zeros(dim)
    return np.mean([embeddings[word] for word in valid_words], axis=0)

def calculate_semantic_similarity(text1, text2, embeddings):
    vec1 = sentence_to_vector(text1, embeddings)
    vec2 = sentence_to_vector(text2, embeddings)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0: return 0.0
    return dot_product / (norm_vec1 * norm_vec2)

# --- 將 GloVe 模型加載到內存 ---
embeddings_index = load_glove_embeddings(glove_file_path)

print("\nGloVe 模型已加載，核心函數已定義完畢。")

Cell 4: 計算語義相似度特徵

In [ ]:
# ==============================================================================
# Cell 4: 計算語義相似度特徵
# 說明：此 Cell 使用上一步加載的模型來為 DataFrame 計算新的特徵欄位。
# ==============================================================================
print("--- Cell 4: 開始計算 'semantic_similarity' 特徵 ---")

# 檢查 'df' 是否存在，確保執行順序正確
if 'df' not in locals() or df.empty:
    raise NameError("錯誤：找不到 DataFrame 'df'。請先執行 Cell 2。")

if 'semantic_similarity' not in df.columns:
    df['semantic_similarity'] = df.progress_apply(
        lambda row: calculate_semantic_similarity(row['response_a'], row['response_b'], embeddings_index),
        axis=1
    )
    print("\n'semantic_similarity' 特徵計算完成。")
else:
    print("\n'semantic_similarity' 特徵已存在，跳過計算。")

display(df[['response_a', 'response_b', 'resp_jaccard', 'semantic_similarity']].head(3))

Cell 5: 對比分析、視覺化與結論

In [ ]:
# ==============================================================================
# Cell 5: 對比分析、視覺化與結論
# 說明：此 Cell 透過圖表和數據，清晰地對比兩個特徵的有效性。
# ==============================================================================
print("--- Cell 5: 開始進行特徵對比分析 ---")

features_to_compare = {
    'resp_jaccard': '回應間的字元級 Jaccard 相似度 (您的現有特徵)',
    'semantic_similarity': '回應間的語義相似度 (基於 GloVe 的新特徵)'
}

for key, name in features_to_compare.items():
    # 1. 繪製小提琴圖
    plt.figure(figsize=(12, 7))
    sns.violinplot(x='label', y=key, data=df, cut=0, inner='quartile')
    plt.title(f'分析：{name} vs. 最終結果', fontsize=16, pad=20)
    plt.xticks(ticks=[0, 1, 2], labels=['A 勝 (label=0)', 'B 勝 (label=1)', '平手 (label=2)'])
    plt.xlabel("最終結果標籤", fontsize=12)
    plt.ylabel("相似度分數", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # 2. 打印統計數據
    print(f"--- 數字統計：{name} ---")
    description = df.groupby('label')[[key]].describe().round(4)
    print(description)
    print("-" * 80 + "\n")